In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tff.backends.reference.set_reference_context()

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import activations

import math
import pickle
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

import seaborn as sns
sns.set_style("whitegrid")

In [4]:
origTra = open('../Datasets/pendigits/pendigits-orig.tra', 'r').read()

In [5]:
listOrigTra = []
for i in origTra.split('COMMENT ')[1:]:
    listOrigTra.append(i.split()[1])
    
len(listOrigTra)

7494

In [6]:
tra = open('../Datasets/pendigits/pendigits.tra', 'r').read()
listTra = []
for i in tra.split('\n'):
    listTra.append(i[-1])
    
len(listTra)

7494

In [7]:
origTes = open('../Datasets/pendigits/pendigits-orig.tes', 'r').read()

In [8]:
listOrigTes = []
for i in origTes.split('COMMENT ')[1:]:
    listOrigTes.append(i.split()[1])
    
len(listOrigTes)

3498

In [9]:
listOrigTes = [str(int(i)+30) for i in listOrigTes]

In [10]:
tes = open('../Datasets/pendigits/pendigits.tes', 'r').read()
listTes = []
for i in tes.split('\n'):
    listTes.append(i[-1])
    
len(listTes)

3498

In [11]:
columns = ['Input1', 'Input2', 'Input3', 'Input4', 'Input5', 'Input6', 'Input7', 'Input8', 'Input9', 'Input10', 'Input11', 'Input12', 'Input13', 'Input14', 'Input15', 'Input16', 'digit']

In [12]:
df_tra = pd.read_csv('../Datasets/pendigits/pendigits.tra', delimiter = ',', names=columns , header=None)
df_tra.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


In [13]:
df_tra_w = df_tra.copy()
df_tra_w['writer'] = listOrigTra
df_tra_w.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8,12
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2,26
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1,11
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4,14
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1,10


In [14]:
df_tes = pd.read_csv('../Datasets/pendigits/pendigits.tes', delimiter = ',', names=columns , header=None)
df_tes.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,88,92,2,99,16,66,94,37,70,0,0,24,42,65,100,100,8
1,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8
2,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8
3,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9
4,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9


In [15]:
df_tes_w = df_tes.copy()
df_tes_w['writer'] = listOrigTes
df_tes_w.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,88,92,2,99,16,66,94,37,70,0,0,24,42,65,100,100,8,36
1,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8,42
2,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8,35
3,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9,37
4,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9,35


In [16]:
df = pd.concat([df_tra_w,df_tes_w])
df.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8,12
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2,26
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1,11
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4,14
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1,10


In [17]:
def getClients(data):
    arr = []
    no_of_writers = len(set(data['writer']))
    for i in range(no_of_writers):
        arr.append([])
    for index, row in data.iterrows():
        rowA = row.to_numpy()
        writer = int(rowA[-1])
        data = rowA[:17]
        arr[writer-1].append(data)
    return arr

In [18]:
def get_batches(HOLDER):
    federated_data = []
    for i in HOLDER:
        client = []
        for index in range(len(i)//20):
            X= []
            Y= []
            for elements in i[index*20:index*20+20]:
                X.append(np.array([np.float32(elements[0]),
                                   np.float32(elements[1]),
                                   np.float32(elements[2]),
                                   np.float32(elements[3]),
                                   np.float32(elements[4]),
                                   np.float32(elements[5]),
                                   np.float32(elements[6]),
                                   np.float32(elements[7]),
                                   np.float32(elements[8]),
                                   np.float32(elements[9]),
                                   np.float32(elements[10]),
                                   np.float32(elements[11]),
                                   np.float32(elements[12]),
                                   np.float32(elements[13]),
                                   np.float32(elements[14]),
                                   np.float32(elements[15])]))
                Y.append(np.array(np.int32(elements[-1])))
            client.append({
                'x':np.array(X),
                'y':np.array(Y)
            })
        federated_data.append(client)
    return federated_data

In [19]:
# federated_train_data = getClients(df_tra_w)
# federated_test_data = getClients(df_tes_w)

In [20]:
# federated_train_data =  get_batches(train)
# federated_validation_data =  get_batches(validation)
# federated_test_data =  get_batches(test)

In [21]:
def getModel(shape):
    return tf.keras.models.Sequential([
        tf.keras.layers.Dense(shape, activation='relu'),
        tf.keras.layers.Dense(10,activation="sigmoid")
    ])

In [25]:
writer_data = df_tra_w.groupby(df_tra_w.writer)

writer_models = []
for i in range(30):
    writer = str(i+1)
    df = writer_data.get_group(writer)
    labels = df['digit'].to_numpy()
    df = df.drop(['digit','writer'], axis=1)
    model = getModel(df.shape[0])
    print("*************************************")
    print(df.shape[0])
    print("*************************************")
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.fit(df.to_numpy(), labels, epochs=5)
    writer_models.append(model)

*************************************
249
*************************************
Epoch 1/5
8/8 [==============================] - 0s 2ms/step - loss: 20.9123 - accuracy: 0.1856
Epoch 2/5
8/8 [==============================] - 0s 2ms/step - loss: 6.4449 - accuracy: 0.4786
Epoch 3/5
8/8 [==============================] - 0s 2ms/step - loss: 2.5042 - accuracy: 0.6691
Epoch 4/5
8/8 [==============================] - 0s 2ms/step - loss: 0.8521 - accuracy: 0.8258
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 0.5674 - accuracy: 0.9026
*************************************
250
*************************************
Epoch 1/5
8/8 [==============================] - 0s 2ms/step - loss: 20.6350 - accuracy: 0.1237
Epoch 2/5
8/8 [==============================] - 0s 1ms/step - loss: 3.3444 - accuracy: 0.5301
Epoch 3/5
8/8 [==============================] - 0s 2ms/step - loss: 0.9486 - accuracy: 0.8081
Epoch 4/5
8/8 [==============================] - 0s 2ms/step - loss: 0.5854 - 

8/8 [==============================] - 0s 2ms/step - loss: 0.1738 - accuracy: 0.9626
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 0.0697 - accuracy: 0.9774
*************************************
250
*************************************
Epoch 1/5
8/8 [==============================] - 0s 2ms/step - loss: 13.7045 - accuracy: 0.1955
Epoch 2/5
8/8 [==============================] - 0s 2ms/step - loss: 1.2380 - accuracy: 0.7565
Epoch 3/5
8/8 [==============================] - 0s 2ms/step - loss: 0.1567 - accuracy: 0.9423
Epoch 4/5
8/8 [==============================] - 0s 2ms/step - loss: 0.0969 - accuracy: 0.9697
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 0.0297 - accuracy: 0.9936
*************************************
247
*************************************
Epoch 1/5
8/8 [==============================] - 0s 2ms/step - loss: 38.0617 - accuracy: 0.1412
Epoch 2/5
8/8 [==============================] - 0s 2ms/step - loss: 11.5934 - accuracy:

8/8 [==============================] - 0s 2ms/step - loss: 3.0033 - accuracy: 0.4798
Epoch 3/5
8/8 [==============================] - 0s 2ms/step - loss: 1.1445 - accuracy: 0.8296
Epoch 4/5
8/8 [==============================] - 0s 2ms/step - loss: 0.5559 - accuracy: 0.9051
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 0.2392 - accuracy: 0.9598


In [32]:
for model in writer_models:
    print(print("*************************************"))
    for layer in model.layers: 
        print(layer.get_config())

*************************************
None
{'name': 'dense_60', 'trainable': True, 'dtype': 'float32', 'units': 249, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'dense_61', 'trainable': True, 'dtype': 'float32', 'units': 10, 'activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
*************************************
None
{'name': 'dense_62', 'trainable': True, 'dtype': 'float32', 'units': 250, 'activation': 'relu', 'use_bias': True, 'kernel_initialize